# Optimizacion de los flujos de trabajo en un almacen con Q-Learning


## Instalamos los paquetes

In [ ]:
# Instalamos numpy
#!pip install numpy

## Importacion de las librerias


In [1]:
import numpy as np

# Problema a Resolver

El problema a resolver será optimizar los flujos dentro del siguiente almacén:

![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Warehouse_1.jpg)


El almacén pertenece a una empresa online minorista que vende productos a una variedad de clientes. Dentro de este almacén, los productos se almacenan en 12 ubicaciones diferentes, etiquetadas con las siguientes letras de la A a la L:


![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Warehouse_2.jpg)

A medida que los clientes hacen los pedidos online, un robot de almacén autónomo se mueve por el almacén para recoger los productos para futuras entregas. Así es como se ve:

![Robot de Almacen Autónomo](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Warehouse_3.jpg)

Las 12 ubicaciones están conectadas a un sistema informático, que clasifica en tiempo real las prioridades de recolección de productos para estas 12 ubicaciones. Por ejemplo, en un momento específico **t**, devolverá la siguiente clasificación:

Rango de Prioridad | Ubicación
-- | --
1 | G
2 | K
3 | L
4 | J
5 | A
6 | I
7 | H
8 | C
9 | B
10 | D
11 | F
12 | E

La ubicación G tiene prioridad 1, lo que significa que es la máxima prioridad, ya que contiene un producto que debe recogerse y entregarse de inmediato. Nuestro robot de almacén autónomo debe moverse a la ubicación G por la ruta más corta, dependiendo de dónde se encuentre. Nuestro objetivo es construir una IA que regrese esa ruta más corta, donde sea que esté el robot. Pero luego, como vemos, las ubicaciones K y L están en las 3 prioridades principales. Por lo tanto, querremos implementar una opción para que nuestro Robot de almacén autónomo pase por algunas ubicaciones intermedias antes de llegar a su ubicación final de máxima prioridad.

La forma en que el sistema calcula las prioridades de las ubicaciones está fuera del alcance de este caso práctico. La razón de esto es que puede haber muchas formas, desde reglas o algoritmos simples, hasta cálculos deterministas y aprendizaje automático. Pero la mayoría de estas formas no serían inteligencia artificial como la conocemos hoy. En lo que realmente queremos centrarnos es en la IA central, que abarca Q-Learning, Deep Q-Learning y otras ramas de Reinforcement Learning. Entonces, solo diremos, por ejemplo, que la ubicación G es la máxima prioridad porque uno de los clientes de platino más leales de la compañía hizo un pedido urgente de un producto almacenado en la ubicación G, que por lo tanto debe entregarse lo antes posible.

Por lo tanto, en conclusión, nuestra misión es construir una IA que siempre tome la ruta más corta a la ubicación de máxima prioridad, sea cual sea la ubicación desde la que comienza, y tener la opción de ir a una ubicación intermedia que se encuentre entre las 3 prioridades principales.

## Configuracion de los parametros gamma y alpha para el algoritmo de Q-Learning


Antes de comenzar con la configuración del algoritmo de Q-Learning es importante tener algunos conceptos claros que nos ayuden a enter un poco mas el proceso para poder resolver el problema propuesto. 

**El valor Q**

Para cada par de estado y acción $(s,a)$, vamos a asociar un valor numérico llamado $Q(s,a)$:
$$Q: (s \in S, a \in A) \mapsto Q(s,a) \in \mathbb{R}$$

Diremos que $Q(s,a)$ es el valor $Q$ de la acción $a$ llevada a cabo en el estado $s$.
Para comprender el propósito de este Valor $Q$, necesitamos introducir la Diferencia Temporal.

**La Diferencia Temporal**

Al principio $t=0$, todos los valores $Q$ se inicializan a 0:

$$\forall s \in S, a \in A, Q(s,a) = 0$$

Ahora supongamos que estamos en el instante $t$, en cierto estado $s_t$. Llevamos a cabo una acción aleatoria $a_t$, que nos lleva al estado $s_{t+1}$ y obtenemos la recompensa $R(s_t,a_t)$.

Ahora podemos presentar la diferencia temporal, que básicamente es el corazón de Q-Learning. La diferencia temporal en el tiempo $t$, denotada por $TDt(s_t,a_t)$, es la diferencia entre:

* $R(s_t,a_t) + \gamma \underset{a}{\max}(Q(s_{t+1},a))$, es decir la recompensa $R(s_t,a_t)$ obtenida al llevar a cabo la acción $a_t$ en el estado $s_t$, más el valor $Q$ de la mejor acción jugada en el estado futuro $s_{t+1}$, descontado por un factor gamma $(\gamma)$, llamado factor de descuento.

* y  $Q(s_t,a_t)$, es decir el valor $Q$ de la acción $a_t$ llevada a cabo en el estado $s_t$, que nos conduce a:

$$TD_t(s_t,a_t) = R(s_t,a_t) + \gamma \underset{a}{\max}(Q(s_{t+1},a)) - Q(s_t,a_t)$$

En ese sentido, la IA repetirá algunas actualizaciones de los valores $Q$ (a través de una ecuación llamada la ecuación de Bellman) hacia diferencias temporales más altas.

En consecuencia, en el siguiente paso final del algoritmo Q-Learning, usamos la diferencia temporal para reforzar los pares (estado, acción) desde el tiempo $t−1$, hasta el tiempo $t$, de acuerdo con la siguiente ecuación:

$$Q_t(s_t,a_t) = Q_{t-1}(s_t,a_t) + \alpha TD_t(s_t,a_t)$$

donde $\alpha \in \mathbb{R}$ es la tasa de aprendizaje, que determina qué tan rápido va el aprendizaje de los valores Q o qué tan grandes son las actualizaciones de los mismos. Su valor suele ser un número real elegido entre 0 y 1, como por ejemplo 0.01, 0.05, 0.1 o 0.5. Cuanto menor sea su valor, más pequeñas serán las actualizaciones de los valores Q y más larga será la ejecución del algoritmo de Q-Learning. Cuanto mayor sea su valor, mayores serán las actualizaciones de los valores Q y más rápido será el algoritmo de Q-Learning.

Esta ecuación anterior es la ecuación de Bellman. Es el pilar fundamental del Q-Learning.

Con este punto de vista, los valores $Q$ miden la acumulación de sorpresa o frustración asociada con el par de acciones y estados $(s_t,a_t)$. En el caso de recibir sorpresa, la IA se refuerza, y en el caso de recibir frustración, la IA se debilita. Por lo tanto, queremos aprender los valores $Q$ que le darán a la IA la máxima buena sorpresa.

En consecuencia, la decisión de qué acción ejecutar depende principalmente del valor $Q$ $Q(s_t,a_t)$. Si la acción $a_t$ ejecutada en el estado $s_t$ está asociada con un valor $Q$ alto $Q(s_t,a_t)$, la IA tendrá una mayor tendencia a elegir la acción $a_t$. Por otro lado, si la acción $a_t$ que se ha llevado a cabo en el estado $s_t$ está asociada con un valor $Q$ pequeño $Q(s_t,a_t)$, la IA tendrá una tendencia menor a elegir la acción $a_t$. Hay varias formas de elegir la mejor acción para ejecutar en cada estado. Primero, cuando estamos en cierto estado $s_t$, simplemente podríamos tomar la acción $a_t$ que maximiza el valor $Q$ $ Q(s_t, a_t)$:

$$a_t = \underset{a}{\textrm{argmax}}(Q(s_t,a))$$

Esta solución es el método **Argmax**.

Otra gran solución, que resulta ser una solución aún mejor para problemas complejos, es el método **Softmax**.

El método **Softmax** consiste en considerar para cada estado $s$ la siguiente distribución:

$$W_s: a \in A \mapsto \frac{\exp(Q(s,a))^{\tau}}{\sum_{a'}\exp(Q(s,a'))^{\tau}} \textrm{ with } \tau \ge 0$$

Luego, elegimos qué acción $a$ llevar a cabo mediante una muestra de un valor aleatorio de esa distribución:

$$a \sim W_s(.)$$


### Tomamos como pseudos hiperparámetros los siguentes valores de Gamma y Alpha 

In [2]:
#El algoritmo se ve muy afectado por el gamma o factor de descuento, para nuestro caso lo hemos colocado con ese valor despues de realizar muchas pruebas.
#Alpha impacta directamente en la velocidad de convergencia, para nuestro caso por no tener tantas dificultades no tenemos riesgo de converger en un mínimo local.
gamma = 0.75
alpha = 0.9

# PARTE 1 - DEFINICION DEL ENTORNO


## Definicion de los estados


Al construir una IA, lo primero que siempre tenemos que hacer es definir el entorno. Y definir un entorno siempre requiere los tres elementos siguientes:

 * Definir los estados
 * Definir las acciones
 * Definir las recompensas

Definamos estos tres elementos, uno por uno.

**Definir los estados.**

Comencemos con los estados. El estado de entrada es simplemente la ubicación donde está nuestro Robot de almacén autónomo en cada momento **t**. Sin embargo, dado que construiremos nuestra IA con ecuaciones matemáticas, codificaremos los nombres de las ubicaciones (A, B, C, …) en números de índice, con respecto a la siguiente asignación:

In [3]:
location_to_state = {'A': 0,
                     'B': 1,
                     'C': 2,
                     'D': 3,
                     'E': 4,
                     'F': 5,
                     'G': 6, 
                     'H': 7, 
                     'I': 8,
                     'J': 9,
                     'K': 10,
                     'L': 11}

Hay una razón específica por la que codificamos los estados con índices del 0 al 11, en lugar de otros enteros. La razón es que trabajaremos con matrices, una matriz de recompensas y una matriz de valores Q, y cada línea y columna de estas matrices corresponderá a una ubicación específica. Por ejemplo, la primera línea de cada matriz, que tiene el índice 0, corresponde a la ubicación A. La segunda línea / columna, que tiene el índice 1, corresponde a la ubicación B. Etc. Veremos el propósito de trabajar con matrices con más detalles. un poco más tarde.

## Definicion de las acciones

Ahora definamos las posibles acciones a realizar Las acciones son simplemente los siguientes movimientos que el robot puede hacer para ir de un lugar a otro. Entonces, por ejemplo, digamos que el robot está en la ubicación J, las posibles acciones que el robot puede llevar a cabo es ir a I, F o K. Y nuevamente, ya que trabajaremos con ecuaciones matemáticas, codificaremos estas acciones con los mismos índices que para los estados. Por lo tanto, siguiendo nuestro mismo ejemplo donde el robot está en la ubicación J en un momento específico, las posibles acciones que el robot puede jugar son, de acuerdo con nuestro mapeo anterior anterior: 5, 8 y 10. De hecho, el índice 5 corresponde a F, el índice 8 corresponde a I y el índice 10 corresponde a K. Por lo tanto, eventualmente, la lista total de acciones que la IA puede llevar a cabo en general es la siguiente:

In [4]:
actions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Obviamente, al estar en una ubicación específica, hay algunas acciones que el robot no puede llevar a cabo. Tomando el mismo ejemplo anterior, si el robot está en la ubicación J, puede ejecutar las acciones 5, 8 y 10, pero no puede ejecutar las otras acciones. Nos aseguraremos de especificar eso al atribuir una recompensa 0 a las acciones que no puede llevar a cabo, y una recompensa 1 a las acciones que si puede realizar. Y eso nos lleva a las recompensas.

## Definicion de las recompensas
## Columnas:    A,B,C,D,E,F,G,H,I,J,K,L

Lo último que tenemos que hacer ahora para construir nuestro entorno es definir un sistema de recompensas. Más específicamente, tenemos que definir una función de recompensa **R**que toma como entradas un estado **s** y una acción **a**, y devuelve una recompensa numérica que la IA obtendrá al llevar a cabo la acción **a** en el estado **s**:

$$R : (\textrm{state}, \textrm{action}) \mapsto r \in \mathbb{R}$$

Entonces, ¿cómo vamos a construir esa función para nuestro caso práctico? Aquí esto es simple. Dado que hay un número discreto y finito de estados (los índices de 0 a 11), así como un número discreto y finito de acciones (mismos índices de 0 a 11), la mejor manera de construir nuestra función de recompensa R es simplemente hacer una matriz. Nuestra función de recompensa será exactamente una matriz de 12 filas y 12 columnas, donde las filas corresponden a los estados y las columnas corresponden a las acciones. De esa forma, en nuestra función $$R: (s, a) r_in $$, **s** será el índice de la fila de la matriz, **a** será el índice de la columna de matriz, y **r** será la celda de los índices  **(s,a)** en la matriz.

Por lo tanto, lo único que tenemos que hacer ahora para definir nuestra función de recompensa es simplemente llenar esta matriz con las recompensas numéricas. Y como acabamos de decir en el párrafo anterior, lo que tenemos que hacer primero es atribuir, para cada una de las 12 ubicaciones, una recompensa 0 por las acciones que el robot no puede ejecutar, y una recompensa 1 por las acciones que el robot puede llevar a cabo. Al hacer eso para cada una de las 12 ubicaciones, terminaremos con una matriz de recompensas. Vamos a construirlo paso a paso, comenzando con la primera ubicación:

Ubicación A.

Cuando se encuentra en la ubicación A, el robot solo puede ir a la ubicación B. Por lo tanto, dado que la ubicación A tiene el índice 0 (primera fila de la matriz) y la ubicación B tiene el índice 1 (segunda columna de la matriz), la primera fila de la matriz de las recompensas obtendrá un 1 en la segunda columna y un 0 en todas las otras columnas, así:

![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Rewards_Matrix_1.png)

Ubicación B.

Al estar en la ubicación B, el robot solo puede ir a tres ubicaciones diferentes: A, C y F. Dado que B tiene el índice 1 (segunda fila), y A, C, F tienen los índices respectivos 0, 2, 5 (1ra, 3ra. , y sexta columna), entonces la segunda fila de la matriz de recompensas obtendrá un 1 en las columnas 1a, 3a y 6a, y 0 en todas las otras columnas. Por lo tanto obtenemos:

![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Rewards_Matrix_2.jpg)

Ubicación C.

Ocurre lo mismo, la ubicación C (de índice 2) solo está conectada a B y G (de índices 1 y 6), por lo que la tercera fila de la matriz de recompensas es:

![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Rewards_Matrix_3.jpg)

En el resto de ubicaciones…

Al hacer lo mismo para todas las demás ubicaciones, finalmente obtenemos nuestra matriz final de recompensas:

![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Rewards_Matrix_4.jpg)

Acabamos de definir las recompensas. Lo hicimos simplemente construyendo esta matriz de recompensas. Es importante entender que esta es la forma en que definimos el sistema de recompensas cuando hacemos Q-Learning con un número finito de entradas y acciones. En el Caso Práctico 2, veremos que procederemos de manera muy diferente.

Ya casi hemos terminado, lo único que tenemos que hacer es atribuir grandes recompensas a las ubicaciones de mayor prioridad. Esto lo hará el sistema informático que devuelve las prioridades de recolección de productos para cada una de las 12 ubicaciones. Por lo tanto, dado que la ubicación G es la máxima prioridad, el sistema informático actualizará la matriz de recompensas atribuyendo una alta recompensa en la celda $$(G,G)$$:

![title](C:\Users\jdiaz\Desktop\Imagenes_Qlearning\Rewards_Matrix_5.jpg)

In [12]:
# Vease en este ejemplo que colocamos 1000 en la posición (G,G) porque estamos interesados que siempre el robot pase por ese lugar
#              A B C D E F G H I J K L
R = np.array([[0,1,0,0,0,0,0,0,0,0,0,0], # A
              [1,0,1,0,0,1,0,0,0,0,0,0], # B
              [0,1,0,0,0,0,1,0,0,0,0,0], # C
              [0,0,0,0,0,0,0,1,0,0,0,0], # D
              [0,0,0,0,0,0,0,0,1,0,0,0], # E
              [0,1,0,0,0,0,0,0,0,1,0,0], # F
              [0,0,1,0,0,0,1000,1,0,0,0,0], # G
              [0,0,0,1,0,0,1,0,0,0,0,1], # H
              [0,0,0,0,1,0,0,0,0,1,0,0], # I
              [0,0,0,0,0,1,0,0,1,0,1,0], # J
              [0,0,0,0,0,0,0,0,0,1,0,1], # K
              [0,0,0,0,0,0,0,1,0,0,1,0]])# L

Y así es como el sistema de recompensas funcionará con Q-Learning. Atribuimos la recompensa más alta (aquí 1000) a la ubicación de máxima prioridad G. Luego puedes ver en las clases de vídeo del curso cómo podemos atribuir una recompensa más alta a la segunda ubicación de mayor prioridad (ubicación K), para hacer que nuestro robot pase por esto ubicación intermedia de máxima prioridad, optimizando así los flujos de movimiento por el almacén.

# PARTE 2 - CONSTRUCCION DE LA SOLUCION DE IA CON Q-LEARNING


### Iniciacíon de los Q-values

In [17]:
# Implementación del proceso de Q-Learning
def route(starting_location, ending_location):
    R_new = np.copy(R) #Copiamos la matriz hecha manualmente con anterioridad y la pasamos a funcionalidad de numpy
    ending_state = location_to_state[ending_location]
    R_new[ending_state, ending_state] = 1000 #Rango del número de iteraciones
    #Definimos el "Qo" para los estados y las acciones
    Q = np.array(np.zeros([12, 12]))# Matríz para los 12 estados y las 12 acciones
    for i in range(1000):
        current_state = np.random.randint(0, 12) #Estado aleatorio de inicio 
        playable_actions = []#Definimos las acciones ejecutables
        for j in range(12): #Rabgo de Acciones posibles
            if R_new[current_state, j] > 0: #Se debe cumplir la recompensa según la matríz de recompenzas 0 1 pra establecer una posible acción
                playable_actions.append(j)
        next_state = np.random.choice(playable_actions)# Genera un proximo estado a partir de las posibles acciones a ejecutar
        TD = R_new[current_state, next_state] + gamma*Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state] # Calculamos manualmente la Diferencia Temporal TD
        Q[current_state, next_state] = Q[current_state, next_state] + alpha*TD #Actualiza los valores de Q, haciendo uso de la ecuación de Bellman
    

In [18]:
# Obtenemos la siguiente matriz de valores Q finales, es importante destacar que el robot va a priorizar los Q mas altos:
print(Q.astype(int))

[[   0 1674    0    0    0    0    0    0    0    0    0    0]
 [1249    0 2231    0    0 1256    0    0    0    0    0    0]
 [   0 1673    0    0    0    0 2977    0    0    0    0    0]
 [   0    0    0    0    0    0    0 2233    0    0    0    0]
 [   0    0    0    0    0    0    0    0  705    0    0    0]
 [   0 1674    0    0    0    0    0    0    0  939    0    0]
 [   0    0 2221    0    0    0 3968 2227    0    0    0    0]
 [   0    0    0 1675    0    0 2977    0    0    0    0 1676]
 [   0    0    0    0  530    0    0    0    0  939    0    0]
 [   0    0    0    0    0 1257    0    0  705    0 1251    0]
 [   0    0    0    0    0    0    0    0    0  943    0 1667]
 [   0    0    0    0    0    0    0 2233    0    0 1222    0]]


# PARTE 3 - MODELO EN PRODUCCION

Calculamos la ruta óptima desde cualquier ubicación inicial a cualquier ubicación final de máxima prioridad. La idea aquí será implementar una función de ruta, que tomará como entradas la ubicación de inicio donde se encuentra nuestro robot de almacén autónomo en un momento específico y la ubicación de finalización donde tiene que ir con la máxima prioridad, y eso volverá como genera la ruta más corta dentro de una lista. Sin embargo, dado que queremos indicar las ubicaciones con sus nombres (en letras), a diferencia de sus estados (en índices), necesitaremos un diccionario que asigne los estados de ubicaciones (en índices) a los nombres de ubicaciones (en letras). Y eso es lo primero que haremos aquí en esta tercera parte, usando un truco para invertir nuestro diccionario anterior `location\_to\_state`, ya que de hecho simplemente queremos obtener el mapeo inverso exacto de este diccionario:

In [20]:
# Hacer un mapeo de los estados a las ubicaciones
state_to_location = {state: location for location, state in location_to_state.items()}

Aquí es cuando entra en juego la sección de código más importante. Estamos a punto de implementar la función final `route ()` que tomará como entradas las ubicaciones de inicio y finalización, y que devolverá la ruta óptima entre estas dos ubicaciones. Para explicar exactamente qué hará esta función de ruta, enumeremos los diferentes pasos del proceso, al pasar de la ubicación E a la ubicación G:

1. Comenzamos en nuestra ubicación inicial E. 
2. Obtenemos el estado de ubicación E, que según nuestro mapeo location_to_state es $s_0=4$. 
3. En la fila del estado $s_0=4$ de nuestra matriz de valores $Q$, hallamos la columna con el mayor valor $Q$ (705).
4. Esta columna tiene el índice 8, por lo que ejecutamos la acción del índice 8 que nos lleva al siguiente estado $s_{t+1}=8$.
5. Obtenemos la ubicación del estado 8, que según nuestro mapeo `state_to_location` es la ubicación I. Por lo tanto, nuestra próxima ubicación es la ubicación I, que se adjunta a nuestra lista que contiene la ruta óptima global.
6. Repetimos los mismos 5 pasos anteriores desde nuestra nueva ubicación inicial I, hasta llegar a nuestro destino final, la ubicación G.

Por lo tanto, dado que no sabemos cuántas ubicaciones tendremos que atravesar entre las ubicaciones inicial y final, tenemos que hacer un bucle while que repetirá el proceso de 5 pasos descrito anteriormente, y que se detendrá tan pronto como lo hagamos llegar a la ubicación final de máxima prioridad:

In [21]:
# Creamos la función final que devolverá la ruta óptima
def route(starting_location, ending_location): #Desde el inicio al final de la ruta
    route = [starting_location]
    next_location = starting_location
    while (next_location != ending_location):# Nos debe devolver un valor diferente, sino itera para conseguirla hasta el final, 
        starting_state = location_to_state[starting_location]# Estado inicial del robot
        next_state = np.argmax(Q[starting_state,])# Proximo estado
        next_location = state_to_location[next_state]#Proxima localización de un paso a la siguiente, para no iterar infinitamente sino consegui el punto
        route.append(next_location)#Entrega la localización próxima a la ruta que sique el robot
        starting_location = next_location #Varia la posición inicial con posisiciones sucesivas
    return route

Cuando lo probamos para ir de E a G, obtenemos las dos rutas óptimas posibles después de imprimir la ruta final ejecutando el código completo varias veces:

In [23]:
# Imprimir la ruta final
print('Ruta:')
route('E', 'G')# Hacemos la busqueda de la ruta que queremos buscar

Ruta:


['E', 'I', 'J', 'F', 'B', 'C', 'G']